In [ ]:
import pyvista as pv
from labdata.schema import * 
from labdata import chronic_paper as cp
from vvasp.VizClasses import  NeuropixelsChronicHolder, Probe
from vvasp.atlas_utils import Atlas
from one.api import ONE
from pathlib import Path

PIDS = ['920865d3-1581-4e81-bd63-ab2186024564','8d89b7d0-33e7-45c6-9972-d3ad4c1e5f4c'] # IBL sessions with similar trajectories
SAVEDIR = Path().home()
one = ONE()

In [ ]:
plotter = pv.Plotter()
plotter.show()
atlas = Atlas(plotter, min_tree_depth=6, max_tree_depth=8)
atlas.add_atlas_region_mesh('MOp','left')
atlas.add_atlas_region_mesh('CP','left')

prbs = []
for pid in PIDS: # add the acute insertions to the plot
    traj = one.alyx.rest('trajectories','list', provenance='Planned', probe_insertion=pid)
    assert len(traj) == 1
    traj=traj[0]
    mlap = [traj['x'], traj['y']]
    mlapdv = [traj['x'], traj['y'], traj['z']]
    angles = [90-traj['theta'], traj['roll'], traj['phi']-90] # convert to vvasp coordinate system
    #angles=[0, 0, -90]
    p = Probe('NP1',plotter, root_intersection_mesh=atlas.meshes['root'])
    p.drive_probe_from_entry(mlap, angles, traj['depth'])
    p.make_inactive()
    prbs.append(p)
    #p = Probe('NP1',plotter, mlapdv, angles, root_intersection_mesh=atlas.meshes['root'])

# add the chronic implantations to the plot
query = EphysRecording.ProbeSetting() * cp.ChronicInsertion() & dict(subject_name='MM008',
                                                                     probe_num=1)
chronic_insertion = (query * ProbeInsertion).fetch(limit=1, as_dict=True)[0]
print(chronic_insertion)
mlap = [chronic_insertion['insertion_ml'], chronic_insertion['insertion_ap']]
angles = [chronic_insertion['insertion_el'], chronic_insertion['insertion_spin'], chronic_insertion['insertion_az']]
p = Probe('NP1',plotter, root_intersection_mesh=atlas.meshes['root'])
p.drive_probe_from_entry(mlap, angles, chronic_insertion['insertion_depth'])

plotter.isometric_view()


In [ ]:
img = plotter.screenshot(SAVEDIR / 'acute_chronic_comparison_trajectories.png', return_img=True, scale=5)